In [1]:
import pandas as pd
import numpy as np
import psycopg2
from sql import engine
import h3
import requests

pa_data = pd.read_csv('./data/Charging_PA.csv',sep=',')

/usr/local/Caskroom/miniconda/base/envs/sql-practice/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (29,30,32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
charging_df = pd.DataFrame(pa_data)

In [3]:
charging_df.drop(['Start Time Zone','End Time Zone', 'Transaction Date (Pacific Time)',
                'GHG Savings (kg)', 'Gasoline Savings (gallons)','Address 1','City', 
                'State/Province', 'Postal Code', 'Country', 'County', 'System S/N', 
                'Model Number','Org Name','EVSE ID','Energy (kWh)','Plug Type','Currency'], inplace= True, axis= 1)

In [4]:
charging_df.columns = charging_df.columns.str.replace(' ', '_')
charging_df.columns = charging_df.columns.str.lower()
charging_df.columns = charging_df.columns.str.rstrip()

In [5]:
charging_df = charging_df.rename(columns={'total_duration_(hh:mm:ss)': 'total_duration', "charging_time_(hh:mm:ss)": "charging_time"})

In [6]:
# Make datetime columns only for 2019
date_cols = ["start_date", "end_date"]
duration_cols = ["total_duration", "charging_time"]

for col in date_cols:
     charging_df[date_cols] = charging_df[date_cols].apply(pd.to_datetime, errors = "coerce")


for col in duration_cols:
     charging_df[duration_cols] = charging_df[duration_cols].apply(pd.to_timedelta, errors="coerce")

In [7]:
charging_2020 = charging_df.query('start_date >= "2020-01-01" & start_date <= "2020-12-31"')
charging_2019 = charging_df.query('start_date >= "2019-01-01" & start_date <= "2019-12-31"')
charging_2018 = charging_df.query('start_date >= "2018-01-01" & start_date <= "2018-12-31"')
charging_2017 = charging_df.query('start_date >= "2017-01-01" & start_date <= "2017-12-31"')
charging_2016 = charging_df.query('start_date >= "2016-01-01" & start_date <= "2016-12-31"')
charging_2015 = charging_df.query('start_date >= "2015-01-01" & start_date <= "2015-12-31"')
charging_2014 = charging_df.query('start_date >= "2014-01-01" & start_date <= "2014-12-31"')
charging_2013 = charging_df.query('start_date >= "2013-01-01" & start_date <= "2013-12-31"')
charging_2012 = charging_df.query('start_date >= "2012-01-01" & start_date <= "2012-12-31"')
charging_2011 = charging_df.query('start_date >= "2011-01-01" & start_date <= "2011-12-31"')

In [8]:
charging_2019.head()

,station_name,mac_address,start_date,end_date,total_duration,charging_time,port_type,port_number,latitude,longitude,fee,ended_by,plug_in_event_id,driver_postal_code,user_id
191586,PALO ALTO CA / WEBSTER #3,0024:B100:0002:4277,2019-01-01 08:46:00,2019-01-01 10:58:00,0 days 02:11:14,0 days 02:06:39,Level 2,1,37.448273,-122.158234,2.86,Plug Out at Vehicle,1546361212,94601.0,492265.0
191587,PALO ALTO CA / BRYANT #6,0024:B100:0002:6B43,2019-01-01 09:18:00,2019-01-01 10:25:00,0 days 01:07:23,0 days 01:06:51,Level 2,1,37.446667,-122.162170,1.52,Plug Out at Vehicle,2951,94025.0,3174411.0
191588,PALO ALTO CA / BRYANT #1,000D:6F00:009D:9A1B,2019-01-01 09:19:00,2019-01-01 18:36:00,0 days 09:16:18,0 days 03:34:31,Level 2,2,37.446613,-122.162308,2.65,Plug Out at Vehicle,3524,94404.0,2358991.0
191589,PALO ALTO CA / BRYANT #5,0024:B100:0002:6D51,2019-01-01 09:46:00,2019-01-01 11:35:00,0 days 01:49:47,0 days 01:49:26,Level 2,1,37.446690,-122.162231,1.40,Plug Out at Vehicle,2792,94025.0,555879.0
191590,PALO ALTO CA / HIGH #4,0024:B100:0002:1327,2019-01-01 09:54:00,2019-01-01 11:25:00,0 days 01:31:12,0 days 01:30:53,Level 2,2,37.443138,-122.163239,2.08,Plug Out at Vehicle,2416,94305.0,3171851.0


In [9]:
charging_2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47655 entries, 191586 to 239240
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype          
---  ------              --------------  -----          
 0   station_name        47655 non-null  object         
 1   mac_address         47655 non-null  object         
 2   start_date          47655 non-null  datetime64[ns] 
 3   end_date            47655 non-null  datetime64[ns] 
 4   total_duration      47655 non-null  timedelta64[ns]
 5   charging_time       47655 non-null  timedelta64[ns]
 6   port_type           47655 non-null  object         
 7   port_number         47655 non-null  int64          
 8   latitude            47655 non-null  float64        
 9   longitude           47655 non-null  float64        
 10  fee                 47655 non-null  float64        
 11  ended_by            47638 non-null  object         
 12  plug_in_event_id    47655 non-null  int64          
 13  driver_postal_code  46251

In [10]:
# replace NaT in end_date with start_date + total_duration
charging_2019['end_date'] = charging_2019['end_date'].fillna((charging_2019['start_date'] + charging_2019['total_duration']))

<ipython-input-10-7ec4f59bfc9b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['end_date'] = charging_2019['end_date'].fillna((charging_2019['start_date'] + charging_2019['total_duration']))


In [11]:
# create column fee_bol (True if user paid, False if charging was free), drop original fee column
charging_2019['fee_bol'] = np.where(charging_2019['fee']== 0.0, False, True)
charging_2019.drop(['fee'],inplace= True, axis= 1)

<ipython-input-11-99e212b80dbc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['fee_bol'] = np.where(charging_2019['fee']== 0.0, False, True)
/usr/local/Caskroom/miniconda/base/envs/sql-practice/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [12]:
# create colum parking_time to display parking time (plugged in but not charging)
charging_2019['parking_time'] = charging_2019['total_duration'] - charging_2019['charging_time']

<ipython-input-12-e1a37cd582e9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['parking_time'] = charging_2019['total_duration'] - charging_2019['charging_time']


Which year do we want to analyze?

unique identifier for each station plug = combination of plug ID and station name

In [13]:
# create column that displays weekday based on start_date
charging_2019['weekday'] = charging_2019['start_date'].dt.dayofweek

<ipython-input-13-202811ed20f3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['weekday'] = charging_2019['start_date'].dt.dayofweek


In [14]:
charging_2019['weekday'].unique()
mymapp = {'0': 'Monday', '1': 'Tuesday', '2': 'Wednesday', '3': 'Thursday', '4': 'Friday', '5': 'Saturday', '6': 'Sunday'}
charging_2019.applymap(lambda s: mymapp.get(s) if s in mymapp else s)

,station_name,mac_address,start_date,end_date,total_duration,charging_time,port_type,port_number,latitude,longitude,ended_by,plug_in_event_id,driver_postal_code,user_id,fee_bol,parking_time,weekday
191586,PALO ALTO CA / WEBSTER #3,0024:B100:0002:4277,2019-01-01 08:46:00,2019-01-01 10:58:00,0 days 02:11:14,0 days 02:06:39,Level 2,1,37.448273,-122.158234,Plug Out at Vehicle,1546361212,94601.0,492265.0,True,0 days 00:04:35,1
191587,PALO ALTO CA / BRYANT #6,0024:B100:0002:6B43,2019-01-01 09:18:00,2019-01-01 10:25:00,0 days 01:07:23,0 days 01:06:51,Level 2,1,37.446667,-122.162170,Plug Out at Vehicle,2951,94025.0,3174411.0,True,0 days 00:00:32,1
191588,PALO ALTO CA / BRYANT #1,000D:6F00:009D:9A1B,2019-01-01 09:19:00,2019-01-01 18:36:00,0 days 09:16:18,0 days 03:34:31,Level 2,2,37.446613,-122.162308,Plug Out at Vehicle,3524,94404.0,2358991.0,True,0 days 05:41:47,1
191589,PALO ALTO CA / BRYANT #5,0024:B100:0002:6D51,2019-01-01 09:46:00,2019-01-01 11:35:00,0 days 01:49:47,0 days 01:49:26,Level 2,1,37.446690,-122.162231,Plug Out at Vehicle,2792,94025.0,555879.0,True,0 days 00:00:21,1
191590,PALO ALTO CA / HIGH #4,0024:B100:0002:1327,2019-01-01 09:54:00,2019-01-01 11:25:00,0 days 01:31:12,0 days 01:30:53,Level 2,2,37.443138,-122.163239,Plug Out at Vehicle,2416,94305.0,3171851.0,True,0 days 00:00:19,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239236,PALO ALTO CA / WEBSTER #1,0024:B100:0002:4233,2019-12-30 18:57:00,2019-12-30 20:38:00,0 days 01:40:55,0 days 01:07:15,Level 2,1,37.448284,-122.158272,Plug Out at Vehicle,6046,95014.0,2248301,True,0 days 00:33:40,0
239237,PALO ALTO CA / HIGH #2,0024:B100:0002:5959,2019-12-30 18:58:00,2019-12-30 21:06:00,0 days 02:07:33,0 days 01:58:20,Level 2,2,37.443344,-122.163231,CPS Server,6126,94582.0,1691301,True,0 days 00:09:13,0
239238,PALO ALTO CA / BRYANT #3,000D:6F00:009E:D39E,2019-12-30 19:32:00,2019-12-31 07:37:00,0 days 12:05:36,0 days 09:40:19,Level 2,2,37.446583,-122.162109,Plug Out at Vehicle,13879,94301.0,713785,True,0 days 02:25:17,0
239239,PALO ALTO CA / CAMBRIDGE #1,0024:B100:0002:1329,2019-12-30 19:48:00,2019-12-31 08:00:00,0 days 12:11:45,0 days 11:05:21,Level 2,2,37.426155,-122.146065,Plug Out at Vehicle,10318,NaN,NaN,True,0 days 01:06:24,0


In [15]:
def make_weekday_names(row):
    if row['weekday'] == 0:
        return 'Monday'
    if row['weekday'] == 1:
        return 'Tuesday'
    if row['weekday'] == 2:
        return 'Wednesday'
    if row['weekday'] == 3:
        return 'Thursday'
    if row['weekday'] == 4:
        return 'Friday'
    if row['weekday'] == 5:
        return 'Saturday'
    if row['weekday'] == 6:
        return 'Sunday'

In [16]:
charging_2019['weekday'] = charging_2019.apply(lambda row : make_weekday_names(row), axis=1)

<ipython-input-16-2f77aeb8fa5c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['weekday'] = charging_2019.apply(lambda row : make_weekday_names(row), axis=1)


In [17]:
def make_mac_same(row):
    return row['mac_address'].replace(':','')

charging_2019['mac_address'] = charging_2019.apply(lambda row : make_mac_same(row), axis=1)

<ipython-input-17-ed39c61e56b2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['mac_address'] = charging_2019.apply(lambda row : make_mac_same(row), axis=1)


In [18]:
def make_unique_id(row):
    return str(row['mac_address']) + '_' +  str(row['port_number'])

charging_2019['unique_plug_id'] = charging_2019.apply(lambda row : make_unique_id(row), axis=1)

<ipython-input-18-9e905a49289f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['unique_plug_id'] = charging_2019.apply(lambda row : make_unique_id(row), axis=1)


In [19]:
charging_2019['index1'] = charging_2019.index

<ipython-input-19-b41f60fb34ac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['index1'] = charging_2019.index


In [20]:
# change order of columns, put unique_plug_id in front, drop MAC ID
charging_2019 = charging_2019[['station_name',
                            'unique_plug_id', 
                            'start_date',
                            'end_date',
                            'total_duration',
                            'charging_time',
                            'parking_time',
                            'port_type',
                            'port_number',
                            'latitude',
                            'longitude',
                            'ended_by', 
                            'plug_in_event_id',
                            'driver_postal_code', 
                            'user_id',
                            'fee_bol', 
                            'weekday']]

In [21]:
charging_2019 = charging_2019.sort_values(['unique_plug_id', 'start_date'])
charging_2019 = charging_2019.reset_index()
charging_2019.drop(['index'], inplace= True, axis= 1)
charging_2019['idle_time'] = None

In [22]:
max_index = int(len(charging_2019) -1)

for index, row in charging_2019.iterrows():
    if index >= max_index:
        charging_2019['idle_time'][index] = None
    elif index <= max_index:
        if charging_2019['unique_plug_id'][index+1] != charging_2019['unique_plug_id'][index]:
            charging_2019['idle_time'][index] = None
        elif charging_2019['unique_plug_id'][index+1] == charging_2019['unique_plug_id'][index]:
            charging_2019['idle_time'][index] = charging_2019['start_date'][index+1] - charging_2019['end_date'][index]

<ipython-input-22-f94ba14cd3ef>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['idle_time'][index] = charging_2019['start_date'][index+1] - charging_2019['end_date'][index]
<ipython-input-22-f94ba14cd3ef>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['idle_time'][index] = None
<ipython-input-22-f94ba14cd3ef>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['idle_time'][index] = None


In [23]:
duration_cols = ["idle_time"]

for col in duration_cols:
     charging_2019[duration_cols] = charging_2019[duration_cols].apply(pd.to_timedelta, errors="coerce")

In [24]:
# bring idle_time to front
charging_2019 = charging_2019[['station_name',
                            'unique_plug_id', 
                            'start_date',
                            'end_date',
                            'total_duration',
                            'charging_time',
                            'parking_time',
                            'idle_time',
                            'port_type',
                            'port_number',
                            'latitude',
                            'longitude',
                            'ended_by', 
                            'plug_in_event_id',
                            'driver_postal_code', 
                            'user_id',
                            'fee_bol', 
                            'weekday']]

In [25]:
# make column whether zip is from PA (1) or not (0)
pa_zip_codes = ['94020', '94301', '94306', '94022', '94302', '95033', '94024', '94303', '94028', '94304']
def determine_pa_zip(row):
    if str(row['driver_postal_code']) in pa_zip_codes:
        return 1
    elif str(row['driver_postal_code']) not in pa_zip_codes:
        return 0
charging_2019['pa_zip_code'] = charging_2019.apply(lambda row: determine_pa_zip(row), axis=1)

In [26]:
charging_2019.pa_zip_code.nunique()

1

In [27]:
# determine local & out-of-city users 
# first get rid of decimal in postal code
charging_2019['driver_postal_code'] = charging_2019['driver_postal_code'].fillna(0).astype(int)
charging_2019['driver_postal_code'].nunique() # 571 unique postal codes

571

In [28]:
def make_h3_code(row):
    return h3.geo_to_h3(
            lat=row['latitude'],
            lng=row['longitude'],
            resolution=8    # Average Hexagon Area (km2): 0.7373276
                            # Average Hexagon Edge Length (km)
                            # https://h3geo.org/docs/core-library/restable/
)
charging_2019['h3_code'] = charging_2019.apply(lambda row : make_h3_code(row), axis=1)
charging_2019.h3_code.nunique()

5

In [29]:
# table_name = 'charging_df_nd_2019'
# # Write records stored in a dataframe to SQL database
# if engine!=None:
#     try:
#         charging_2019.to_sql(name=table_name, # Name of SQL table
#                         con=engine, # Engine or connection
#                         if_exists='replace', # Drop the table before inserting new values 
#                         index=False, # Write DataFrame index as a column
#                         chunksize=5000, # Specify the number of rows in each batch to be written at a time
#                         method='multi') # Pass multiple values in a single INSERT clause
#         print(f"The {table_name} table was imported successfully.")
#     # Error handling
#     except (Exception, psycopg2.DatabaseError) as error:
#         print(error)
#         engine = None

### Potential Research Questions

Goal: Incentivize consumers in Germany (MUC and surroundings) to buy an EV by providing an optimal charging infrastructure (this means: there is (almost) always a charging station available at the right place (residential, leisure, work)

Identifying charging behavior patterns in relation to location specifics - Is Palo Alto a best practice example? If yes, what can other cities (e.g. MUC) learn from them in terms of charging infrastructure?

* Impact of COVID -- if time permits
* crisis 2013/2014 -- if time permits
* identify usage patterns according to location 
    * What are peak hours? Difference between weekdays / weekends?, per month? per year? 
    * peak times: How many PA users and how many from outside?
    * avg. charging time, avg. parking time, idle times
    * How many users are not from PA? What does MUC have to take into account in terms of "outside"/"shuttle" users?
    * Analyze recurring charging customers with zip codes not from PA? Which stations do they use? 
* What infrastructure strategy is PA using (e.g. how many new stations per year, where? (location characteristics); Is Palo Alto best practice? (define KPIs, what makes a best practice? Ideas: high idle times (even during peak hours or at least other free station nearby), high ratio of charging stations per EV, high coverage?)
* How does MUC compare?
* Which stations were added newly each year? compare with MUC

Potential categories: living, working, shopping, recreation

In [30]:
# h3 to geo returns a tuple, so we have to convert it into a list to access it right

h3_list = charging_2019.h3_code.unique()
geo_center_list = []

for i in h3_list:
    k = list(h3.h3_to_geo(i))
    geo_center_list.append(k)


In [31]:
geo_center_list_string = []
def make_string_geo():
    for i in geo_center_list:
        string = str(i[0]) + ',' + str(i[1])
        geo_center_list_string.append(string)
make_string_geo()
print(geo_center_list_string)

['37.44834466008173,-122.1573800283881', '37.44482864275438,-122.13964995071379', '37.421278892391406,-122.11027985263733', '37.44009132316512,-122.16042161897487', '37.42832113563978,-122.14573587238898']


In [32]:
api_key= 'Z1XB-w87ksYmy5gJnj4gfLPJUJRkiyyPDLEtyjnMYy0'
# url_range = f'https://browse.search.hereapi.com/v1/browse?at={GEO}&in=circle:{GEO};r=700&limit=100&apiKey={api_key}'

title_list = []
cat_id_list = []
cat_def_list = []
cat_name_list = []
lat_list = []
lng_list = []
source_geo = []
h3_source_list = []


count = 0

for index, GEO in enumerate(geo_center_list_string):
    resp = requests.get(url = f'https://browse.search.hereapi.com/v1/browse?at={GEO}&in=circle:{GEO};r=700&limit=100&apiKey={api_key}')
    data= resp.json()

    for i in data['items']:
        if 'categories' not in i:
            continue
        else:
            title_list.append(str(i['title']))
            cat_id_list.append(str(i['categories'][0]['id']))
            cat_name_list.append(str(i['categories'][0]['name']))
            lat_list.append(str(i['position']['lat']))
            lng_list.append(str(i['position']['lng']))
            source_geo.append(GEO)
            h3_source_list.append(h3_list[index])
            count +=1
print('Places nearby: ' + str(count))

Places nearby: 427


In [33]:
dict = {'title': title_list, 'cat_id': cat_id_list, 'cat_name': cat_name_list, 'latitude': lat_list, 'longitude': lng_list, 'geo_source': source_geo, 'h3_source': h3_source_list} 
cat_df = pd.DataFrame(dict)

In [34]:
# searched for 1 instead of 100 because we had null-values in cat 500 ( there was a 550 at the beginning)

def make_cat_def(row):
    if row['cat_id'][:1] == '1':
        return 'Eat & Drink'
    elif row['cat_id'][:1] == '2':
        return 'Going Out-Entertainment'
    elif row['cat_id'][:1] == '3':
        return 'Sights and Museums'
    elif row['cat_id'][:1] == '4':
        return 'Transport'
    elif row['cat_id'][:1] == '5':
        return 'Accommodations'
    elif row['cat_id'][:1] == '6':
        return 'Shopping'
    elif row['cat_id'][:1] == '7':
        return 'Business and Services'
    elif row['cat_id'][:1] == '8':
        return 'Facilities'
    elif row['cat_id'][:1] == '9':
        return 'Areas and Buildings'

cat_df['cat_definition'] = cat_df.apply(lambda row : make_cat_def(row), axis=1)

In [35]:
display(cat_df.cat_definition.unique())

# No Cat 9 ( Areas and Buildings )

array(['Business and Services', 'Shopping', 'Facilities', 'Eat & Drink',
       'Accommodations', 'Sights and Museums', 'Transport',
       'Going Out-Entertainment'], dtype=object)

In [36]:
cat_df.head()

,title,cat_id,cat_name,latitude,longitude,geo_source,h3_source,cat_definition
0,Nokia Growth Partners,700-7250-0136,Business Facility,37.44841,-122.1574,"37.44834466008173,-122.1573800283881",88283475d9fffff,Business and Services
1,Slalom Silicon Valley,700-7200-0264,Management and Consulting Services,37.44841,-122.1574,"37.44834466008173,-122.1573800283881",88283475d9fffff,Business and Services
2,Just Terri,600-6950-0000,Hair & Beauty,37.4481,-122.15772,"37.44834466008173,-122.1573800283881",88283475d9fffff,Shopping
3,Lan Liu Bowling - Keller Williams Realty,700-7400-0145,Real Estate Services,37.4481,-122.15772,"37.44834466008173,-122.1573800283881",88283475d9fffff,Business and Services
4,Susan K Sutherland Certified Public Accountant,700-7200-0271,Finance and Insurance,37.44831,-122.15688,"37.44834466008173,-122.1573800283881",88283475d9fffff,Business and Services


In [37]:
cat_df = cat_df[[   'h3_source',
                    'cat_definition', 
                    'cat_name',
                    'title'
                ]]

In [38]:
cat_df.sort_values(['h3_source', 'cat_definition'])

,h3_source,cat_definition,cat_name,title
240,8828347433fffff,Business and Services,Engineering and Scientific Services,Fergus Garber Young Architects
241,8828347433fffff,Business and Services,Social Services,Bay Area Senior Games
242,8828347433fffff,Business and Services,"Advertising/Marketing, PR & Market Research",Opportunity Health
243,8828347433fffff,Business and Services,Consumer Services,Peninsula Healthcare Connection
252,8828347433fffff,Business and Services,Consumer Services,Bhati Santosh MD Palo Alto Med FDTN
...,...,...,...,...
77,88283475d9fffff,Facilities,Therapist,Gloria Kardong MD
78,88283475d9fffff,Facilities,Therapist,"Renee Burgard, LCSW"
87,88283475d9fffff,Facilities,Parking Garage/Parking House,Webster-Cowper Parking Garage
89,88283475d9fffff,Facilities,Family/General Practice Physicians,"Dr. Richard D. Lee, MD"


In [39]:
group_cat =  cat_df.groupby(['h3_source', 'cat_definition']).count().sort_values(['h3_source','cat_name'], ascending=False)    

# charging_2019 = charging_2019.sort_values(['unique_plug_id', 'start_date'])
            

In [40]:
# category = {
#     100: 'Eat & Drink', #The Eat and Drink category is a top level category for places where food or beverages are prepared or served.
#     200: 'Going Out-Entertainment', #The Going Out-Entertainment category is a top level category for places commonly associated with entertainment, such as bars, cinemas, theatres, casinos and night clubs.
#     300: 'Sights and Museums', #The Sights and Museums category is a top level category for places of special interest, such as common tourist attractions, museums and places of worship.
#     400: 'Transport', #The Transport category is a top level category for places commonly associated with pedestrian and cargo transport facilities, including airports, rail yards and seaports.
#     500: 'Accommodations', #The Accommodations category is a top level category for places offering lodging accommodations, dwellings or similar living quarters to travellers, such as hotels, motels, resorts, cruise ships and campgrounds.
#     600: 'Shopping', #The Shopping category is a top level category for places where consumer goods are commonly sold, such as clothing stores, grocery stores, hardware stores and other types of shopping centers.
#     700: 'Business and Services', #The Business and Services category is a top level category for places that provide professional services to other businesses, such as printing, photocopying, graphic design, marketing, advertising and other general business services.
#     800: 'Facilities', #The Facilities category is a top level category for places associated with specialized facilities, such as sports venues, government buildings, health care centers and other types of facilities.
#     900: 'Areas and Buildings', #The Areas and Buildings category is a top level category for places that are owned, operated or managed by municipalities, such as cities, towns, villages, boroughs and shires.
# }
display(group_cat)

cat_name  title
h3_source       cat_definition                          
88283475d9fffff Business and Services          68     68
                Facilities                     30     30
                Eat & Drink                     1      1
                Shopping                        1      1
88283475cbfffff Business and Services          21     21
                Facilities                     16     16
                Shopping                        6      6
                Sights and Museums              4      4
                Accommodations                  2      2
                Eat & Drink                     1      1
                Transport                       1      1
8828347553fffff Facilities                     29     29
                Business and Services          21     21
                Shopping                        9      9
                Eat & Drink                     8      8
                Transport                       8      8
                Sights and Museums              3      3
                Going Out-Entertainment         2      2
                Accommodations                  1      1
882834743dfffff Business and Services          46     46
                Facilities                     31     31
                Shopping                        8      8
                Eat & Drink                     7      7
                Sights and Museums              2      2
                Going Out-Entertainment         1      1
8828347433fffff Facilities                     92     92
                Business and Services           7      7
                Shopping                        1      1

In [41]:
charging_2019.head()

,station_name,unique_plug_id,start_date,end_date,total_duration,charging_time,parking_time,idle_time,port_type,port_number,latitude,longitude,ended_by,plug_in_event_id,driver_postal_code,user_id,fee_bol,weekday,pa_zip_code,h3_code
0,PALO ALTO CA / BRYANT #1,000D6F00009D9A1B_1,2019-02-01 11:55:00,2019-02-01 12:41:00,0 days 00:46:11,0 days 00:44:25,0 days 00:01:46,10 days 00:23:00,Level 1,1,37.446613,-122.162308,Customer,3604,95008,265179.0,True,Friday,0,88283475d9fffff
1,PALO ALTO CA / BRYANT #1,000D6F00009D9A1B_1,2019-02-11 13:04:00,2019-02-11 16:03:00,0 days 02:58:07,0 days 02:53:48,0 days 00:04:19,21 days 07:19:00,Level 1,1,37.446613,-122.162308,Customer,3626,94303,208307.0,True,Monday,0,88283475d9fffff
2,PALO ALTO CA / BRYANT #1,000D6F00009D9A1B_1,2019-03-04 23:22:00,2019-03-04 23:24:00,0 days 00:02:14,0 days 00:01:19,0 days 00:00:55,6 days 11:17:00,Level 1,1,37.446613,-122.162308,Customer,3678,94025,705633.0,False,Monday,0,88283475d9fffff
3,PALO ALTO CA / BRYANT #1,000D6F00009D9A1B_1,2019-03-11 10:41:00,2019-03-11 12:54:00,0 days 02:13:45,0 days 02:09:49,0 days 00:03:56,241 days 22:59:00,Level 1,1,37.446613,-122.162308,Customer,3693,93001,2589921.0,True,Monday,0,88283475d9fffff
4,PALO ALTO CA / BRYANT #1,000D6F00009D9A1B_1,2019-11-08 11:53:00,2019-11-08 12:59:00,0 days 01:06:11,0 days 01:04:05,0 days 00:02:06,NaT,Level 1,1,37.446613,-122.162308,Customer,4254,94303,208307,True,Friday,0,88283475d9fffff


In [42]:
charging_name_h3 = charging_2019[['h3_code','station_name', 'unique_plug_id']]

In [44]:
sort_cat =  charging_name_h3.sort_values(['h3_code'], ascending=False)
sort_cat['station_count'] = sort_cat.station_name
sort_cat['plug_count'] = sort_cat.unique_plug_id
sort_cat['h3_source'] = sort_cat.h3_code

sort_cat = sort_cat[['h3_source', 'station_count', 'plug_count']]


charging_name_h3 =  sort_cat.groupby(['h3_source']).nunique()
charging_name_h3
display(charging_name_h3)

,station_count,plug_count
h3_source,,
8828347433fffff,5,10
882834743dfffff,9,22
8828347553fffff,6,8
88283475cbfffff,3,5
88283475d9fffff,10,20


In [55]:
# merged_dfs = pd.merge(group_cat, charging_name_h3, on='h3_source')

merged_dfs = group_cat.join(charging_name_h3, on = 'h3_source')
#pd.join(group_cat, charging_name_h3, on='h3_source')

In [63]:
merged_dfs['total_poi'] = merged_dfs.groupby(['h3_source']).sum()

In [65]:
merged_dfs.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 28 entries, ('88283475d9fffff', 'Business and Services') to ('8828347433fffff', 'Shopping')
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   cat_name       28 non-null     int64  
 1   title          28 non-null     int64  
 2   station_count  28 non-null     int64  
 3   plug_count     28 non-null     int64  
 4   total_poi      0 non-null      float64
dtypes: float64(1), int64(4)
memory usage: 1.5+ KB


In [ ]:
nice_merge_df =  merged_dfs.groupby(['h3_source','station_count','plug_count']).count().sort_values(['h3_source','cat_name'], ascending=False) 

In [ ]:
display(nice_merge_df)

In [102]:
new_df_dict = { 
                'h3_code': h3_list, 
                'station_count': None, 
                'plug_count': None, 
                'Eat_and_Drink': None, 
                'Going_Out-Entertainment': None, 
                'Sights_and_Museums': None, 
                'Transport': None,
                'Accommodations': None, 
                'Shopping': None, 
                'Business_and_Services': None,
                'Facilities': None, 
                'Areas_and_Buildings': None
}

In [103]:
master_compare_df = pd.DataFrame(new_df_dict)

In [104]:
display(cat_df)


,h3_source,cat_definition,cat_name,title
0,88283475d9fffff,Business and Services,Business Facility,Nokia Growth Partners
1,88283475d9fffff,Business and Services,Management and Consulting Services,Slalom Silicon Valley
2,88283475d9fffff,Shopping,Hair & Beauty,Just Terri
3,88283475d9fffff,Business and Services,Real Estate Services,Lan Liu Bowling - Keller Williams Realty
4,88283475d9fffff,Business and Services,Finance and Insurance,Susan K Sutherland Certified Public Accountant
...,...,...,...,...
422,882834743dfffff,Eat & Drink,Restaurant,Domino's
423,882834743dfffff,Business and Services,Attorney,Lumen Patent Firm
424,882834743dfffff,Business and Services,Real Estate Services,Yale Investment Company
425,882834743dfffff,Business and Services,Attorney,Peter F Goldscheider Attorney at Law


In [ ]:
# category = {
#     100: 'Eat & Drink', #The Eat and Drink category is a top level category for places where food or beverages are prepared or served.
#     200: 'Going Out-Entertainment', #The Going Out-Entertainment category is a top level category for places commonly associated with entertainment, such as bars, cinemas, theatres, casinos and night clubs.
#     300: 'Sights and Museums', #The Sights and Museums category is a top level category for places of special interest, such as common tourist attractions, museums and places of worship.
#     400: 'Transport', #The Transport category is a top level category for places commonly associated with pedestrian and cargo transport facilities, including airports, rail yards and seaports.
#     500: 'Accommodations', #The Accommodations category is a top level category for places offering lodging accommodations, dwellings or similar living quarters to travellers, such as hotels, motels, resorts, cruise ships and campgrounds.
#     600: 'Shopping', #The Shopping category is a top level category for places where consumer goods are commonly sold, such as clothing stores, grocery stores, hardware stores and other types of shopping centers.
#     700: 'Business and Services', #The Business and Services category is a top level category for places that provide professional services to other businesses, such as printing, photocopying, graphic design, marketing, advertising and other general business services.
#     800: 'Facilities', #The Facilities category is a top level category for places associated with specialized facilities, such as sports venues, government buildings, health care centers and other types of facilities.
#     900: 'Areas and Buildings', #The Areas and Buildings category is a top level category for places that are owned, operated or managed by municipalities, such as cities, towns, villages, boroughs and shires.
# }

In [105]:
def count_station_in_h3(row):
    return charging_2019[charging_2019.h3_code == row['h3_code']].nunique()['station_name']  
def count_plug_id_in_h3(row):
    return charging_2019[charging_2019.h3_code == row['h3_code']].nunique()['unique_plug_id']
def count_100_in_h3(row):
    return cat_df[(cat_df.h3_source == row['h3_code']) & (cat_df.cat_definition == 'Eat & Drink')].count()
def count_200_in_h3(row):
    return cat_df[(cat_df.h3_source == row['h3_code']) & (cat_df.cat_definition == 'Going Out-Entertainment')].count()
def count_300_in_h3(row):
    return cat_df[(cat_df.h3_source == row['h3_code']) & (cat_df.cat_definition == 'Sights and Museums')].count()
def count_400_in_h3(row):
    return cat_df[(cat_df.h3_source == row['h3_code']) & (cat_df.cat_definition == 'Transport')].count()
def count_500_in_h3(row):
    return cat_df[(cat_df.h3_source == row['h3_code']) & (cat_df.cat_definition == 'Accommodations')].count()
def count_600_in_h3(row):
    return cat_df[(cat_df.h3_source == row['h3_code']) & (cat_df.cat_definition == 'Shopping')].count()
def count_700_in_h3(row):
    return cat_df[(cat_df.h3_source == row['h3_code']) & (cat_df.cat_definition == 'Business and Services')].count()
def count_800_in_h3(row):
    return cat_df[(cat_df.h3_source == row['h3_code']) & (cat_df.cat_definition == 'Facilities')].count()
def count_900_in_h3(row):
    return cat_df[(cat_df.h3_source == row['h3_code']) & (cat_df.cat_definition == 'Areas and Buildings')].count()
def total_poi(row):
    return row['Eat_and_Drink']+row['Going_Out-Entertainment']+row['Sights_and_Museums']+row['Transport']+row['Accommodations']+row['Shopping']+row['Business_and_Services']+row['Facilities']+row['Areas_and_Buildings']

In [106]:
master_compare_df['station_count'] = master_compare_df.apply(lambda row : count_station_in_h3(row), axis=1)
master_compare_df['plug_count'] = master_compare_df.apply(lambda row : count_plug_id_in_h3(row), axis=1)
master_compare_df['Eat_and_Drink'] = master_compare_df.apply(lambda row : count_100_in_h3(row), axis=1)
master_compare_df['Going_Out-Entertainment'] = master_compare_df.apply(lambda row : count_200_in_h3(row), axis=1)
master_compare_df['Sights_and_Museums'] = master_compare_df.apply(lambda row : count_300_in_h3(row), axis=1)
master_compare_df['Transport'] = master_compare_df.apply(lambda row : count_400_in_h3(row), axis=1)
master_compare_df['Accommodations'] = master_compare_df.apply(lambda row : count_500_in_h3(row), axis=1)
master_compare_df['Shopping'] = master_compare_df.apply(lambda row : count_600_in_h3(row), axis=1)
master_compare_df['Business_and_Services'] = master_compare_df.apply(lambda row : count_700_in_h3(row), axis=1)
master_compare_df['Facilities'] = master_compare_df.apply(lambda row : count_800_in_h3(row), axis=1)
master_compare_df['Areas_and_Buildings'] = master_compare_df.apply(lambda row : count_900_in_h3(row), axis=1)
master_compare_df['total_poi'] = master_compare_df.apply(lambda row : total_poi(row), axis=1)

In [107]:
display(master_compare_df)

,h3_code,station_count,plug_count,Eat_and_Drink,Going_Out-Entertainment,Sights_and_Museums,Transport,Accommodations,Shopping,Business_and_Services,Facilities,Areas_and_Buildings,total_poi
0,88283475d9fffff,10,20,1,0,0,0,0,1,68,30,0,100
1,88283475cbfffff,3,5,1,0,4,1,2,6,21,16,0,51
2,8828347553fffff,6,8,8,2,3,8,1,9,21,29,0,81
3,8828347433fffff,5,10,0,0,0,0,0,1,7,92,0,100
4,882834743dfffff,9,22,7,1,2,0,0,8,46,31,0,95


In [95]:
display(merged_dfs)

cat_name  title  station_count  \
h3_source       cat_definition                                            
88283475d9fffff Business and Services          68     68             10   
                Facilities                     30     30             10   
                Eat & Drink                     1      1             10   
                Shopping                        1      1             10   
88283475cbfffff Business and Services          21     21              3   
                Facilities                     16     16              3   
                Shopping                        6      6              3   
                Sights and Museums              4      4              3   
                Accommodations                  2      2              3   
                Eat & Drink                     1      1              3   
                Transport                       1      1              3   
8828347553fffff Facilities                     29     29              6   
                Business and Services          21     21              6   
                Shopping                        9      9              6   
                Eat & Drink                     8      8              6   
                Transport                       8      8              6   
                Sights and Museums              3      3              6   
                Going Out-Entertainment         2      2              6   
                Accommodations                  1      1              6   
882834743dfffff Business and Services          46     46              9   
                Facilities                     31     31              9   
                Shopping                        8      8              9   
                Eat & Drink                     7      7              9   
                Sights and Museums              2      2              9   
                Going Out-Entertainment         1      1              9   
8828347433fffff Facilities                     92     92              5   
                Business and Services           7      7              5   
                Shopping                        1      1              5   

                                         plug_count  total_poi  
h3_source       cat_definition                                  
88283475d9fffff Business and Services            20        NaN  
                Facilities                       20        NaN  
                Eat & Drink                      20        NaN  
                Shopping                         20        NaN  
88283475cbfffff Business and Services             5        NaN  
                Facilities                        5        NaN  
                Shopping                          5        NaN  
                Sights and Museums                5        NaN  
                Accommodations                    5        NaN  
                Eat & Drink                       5        NaN  
                Transport                         5        NaN  
8828347553fffff Facilities                        8        NaN  
                Business and Services             8        NaN  
                Shopping                          8        NaN  
                Eat & Drink                       8        NaN  
                Transport                         8        NaN  
                Sights and Museums                8        NaN  
                Going Out-Entertainment           8        NaN  
                Accommodations                    8        NaN  
882834743dfffff Business and Services            22        NaN  
                Facilities                       22        NaN  
                Shopping                         22        NaN  
                Eat & Drink                      22        NaN  
                Sights and Museums               22        NaN  
                Going Out-Entertainment          22        NaN  
8828347433fffff Facilities         

In [ ]:
station_c= charging_2019.station_name.query(f'h3_code == {i}').nunique()
#charging_2020 = charging_df.query('start_date >= "2020-01-01" & start_date <= "2020-12-31"')

In [ ]:
h3_code = h3_source_list

for i in h3_code:
    station_count = charging_2019.query(f'h3_code == {str(i)}').nunique()
    print(station_count)
    break


    

In [58]:
merged_dfs.to_csv('marged_PA.csv', encoding='utf-8', index=True)